In [5]:
import torch
ones = torch.zeros(3, 2) + 1
ones

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

In [6]:
threes=ones*3
threes

tensor([[3., 3.],
        [3., 3.],
        [3., 3.]])

In [7]:
rand=torch.rand(2,4)
rand

tensor([[0.1692, 0.5810, 0.2161, 0.5361],
        [0.0867, 0.5647, 0.9753, 0.9766]])

In [12]:
doublerand=rand*2
print(doublerand.shape)
doublerand

torch.Size([2, 4])


tensor([[0.3384, 1.1619, 0.4323, 1.0723],
        [0.1733, 1.1294, 1.9506, 1.9531]])

In [9]:
print(torch.unique(torch.tensor([1, 2, 1, 2, 1, 2])))

tensor([1, 2])


In [11]:
v1 = torch.tensor([1., 0., 0.])         # x unit vector
v2 = torch.tensor([0., 1., 0.])         # y unit vector
print(torch.cross(v2, v1)) # negative of z unit vector (v1 x v2 == -v2 x v1)
print(v1.shape)

tensor([ 0.,  0., -1.])
torch.Size([3])


In [2]:
import torch
if torch.cuda.is_available():
    gpu_rand = torch.rand(2, 2, device='cuda')
    print(gpu_rand)
else:
    print('Sorry, CPU only.')

tensor([[0.3888, 0.8747],
        [0.9726, 0.3369]], device='cuda:0')


In [4]:
if torch.cuda.is_available():
    my_device = torch.device('cuda')
else:
    my_device = torch.device('cpu')
print('Device: {}'.format(my_device))

x = torch.rand(2, 3, device=my_device)
print(x)

Device: cuda
tensor([[0.0734, 0.3096, 0.5053],
        [0.5882, 0.1422, 0.8458]], device='cuda:0')


If you have an existing tensor living on one device, you can move it to another with the to() method. The following line of code creates a tensor on CPU, and moves it to whichever device handle you acquired in the previous cell.

It is important to know that in order to do computation involving two or more tensors, all of the tensors must be on the same device. 

In [7]:
y = torch.rand(2, 4)
print(y)
y = y.to(my_device)

print(y)

tensor([[0.8509, 0.3518, 0.6497, 0.7260],
        [0.6426, 0.2894, 0.3083, 0.7293]])
tensor([[0.8509, 0.3518, 0.6497, 0.7260],
        [0.6426, 0.2894, 0.3083, 0.7293]], device='cuda:0')


You may only squeeze() dimensions of extent 1. See above where we try to squeeze a dimension of size 2 in c, and get back the same shape we started with. Calls to squeeze() and unsqueeze() can only act on dimensions of extent 1 because to do otherwise would change the number of elements in the tensor.

In [9]:
a = torch.rand(1, 20)
a=a.to(my_device) #move data to GPU
print(a.shape)
print(a)

b = a.squeeze(0)
print(b.shape)
print(b)

torch.Size([1, 20])
tensor([[0.5873, 0.4226, 0.2610, 0.7966, 0.5074, 0.2427, 0.8231, 0.5592, 0.3040,
         0.9747, 0.5377, 0.1433, 0.6375, 0.4927, 0.8456, 0.8243, 0.0516, 0.8355,
         0.7255, 0.6365]], device='cuda:0')
torch.Size([20])
tensor([0.5873, 0.4226, 0.2610, 0.7966, 0.5074, 0.2427, 0.8231, 0.5592, 0.3040,
        0.9747, 0.5377, 0.1433, 0.6375, 0.4927, 0.8456, 0.8243, 0.0516, 0.8355,
        0.7255, 0.6365], device='cuda:0')


In [12]:
batch_me = torch.rand(3, 226, 226)
print(batch_me.shape)
batch_me.unsqueeze_(0)
print(batch_me.shape)

torch.Size([3, 226, 226])
torch.Size([1, 3, 226, 226])


Sometimes you’ll want to change the shape of a tensor more radically, while still preserving the number of elements and their contents. One case where this happens is at the interface between a convolutional layer of a model and a linear layer of the model - this is common in image classification models. A convolution kernel will yield an output tensor of shape features x width x height, but the following linear layer expects a 1-dimensional input. reshape() will do this for you, provided that the dimensions you request yield the same number of elements as the input tensor has:

In [13]:
output3d = torch.rand(6, 20, 20)
print(output3d.shape)

input1d = output3d.reshape(6 * 20 * 20)
print(input1d.shape)

# can also call it as a method on the torch module:
print(torch.reshape(output3d, (6 * 20 * 20,)).shape)

torch.Size([6, 20, 20])
torch.Size([2400])
torch.Size([2400])


When it can, reshape() will return a view on the tensor to be changed - that is, a separate tensor object looking at the same underlying region of memory. This is important: That means any change made to the source tensor will be reflected in the view on that tensor, unless you clone() it.

In [14]:
import numpy as np

numpy_array = np.ones((2, 3))
print(numpy_array)

pytorch_tensor = torch.from_numpy(numpy_array) #convert a numpy array into a tensor
print(pytorch_tensor)

[[1. 1. 1.]
 [1. 1. 1.]]
tensor([[1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)


In [16]:
pytorch_rand = torch.rand(2, 3)
print(pytorch_rand)

numpy_rand = pytorch_rand.numpy()  #convert a tensor into a numpy array
print(numpy_rand)

tensor([[0.6228, 0.9791, 0.9505],
        [0.9391, 0.1913, 0.2905]])
[[0.62279856 0.97910345 0.95046777]
 [0.93909144 0.19129813 0.29045784]]


It is important to know that these converted objects are using the same underlying memory as their source objects, meaning that changes to one are reflected in the other:

In [17]:
numpy_array[1, 1] = 23
print(pytorch_tensor)

pytorch_rand[1, 1] = 17
print(numpy_rand)

tensor([[ 1.,  1.,  1.],
        [ 1., 23.,  1.]], dtype=torch.float64)
[[ 0.62279856  0.97910345  0.95046777]
 [ 0.93909144 17.          0.29045784]]
